In [ ]:
import numpy as np
import pandas as pd
from collections import Counter
import MeCab

In [3]:
train = pd.read_csv("../data/train_Youto_sentence.csv")
m = MeCab.Tagger("-Ochasen -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd")

In [4]:
clue_words = []
for i, v in train.iterrows():
    node = m.parseToNode(v['sentence'])
    clue_word = []
    while node:
        if len(node.surface) is 0:
            node = node.next
            continue
        
        hinshi = node.feature.split(',')
        if hinshi[0] == '名詞' and hinshi[1] == 'サ変接続':
            clue_word.append(node.surface)
        
        node = node.next
        
    clue_words.append(clue_word)

In [5]:
df = pd.DataFrame({"clue_word": clue_words, "label": train.label})

In [6]:
flatten = lambda l: [item for sublist in l for item in sublist if len(item) is not 0]

In [7]:
def word_proba(docs, word):
    word_count = np.array([Counter(doc)[word] for doc in docs])
    return pd.Series(word_count / np.sum(word_count))

def word_entropy(word_proba):
    return np.sum(np.nan_to_num(-word_proba * np.log2(word_proba)))

In [8]:
word_set = set(flatten(df['clue_word'].values))

word_entropy_positive = \
    np.array([df.loc[df.label == 1, 'clue_word'].pipe(word_proba, word).pipe(word_entropy) for word in word_set]) \
    / np.log2(len(df.loc[df.label == 1]))
word_entropy_negative = \
    np.array([df.loc[df.label == 0, 'clue_word'].pipe(word_proba, word).pipe(word_entropy) for word in word_set]) \
    / np.log2(len(df.loc[df.label == 0]))

/usr/local/var/pyenv/versions/3.6.1/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log2
  
/usr/local/var/pyenv/versions/3.6.1/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
word_entropy_df = pd.DataFrame({"clue_word": list(word_set), "entropy_positive": word_entropy_positive, "entropy_negative": word_entropy_negative})
word_entropy_df

,clue_word,entropy_positive,entropy_negative
0,併発,0.094660,0.000000
1,緩衝,0.219793,0.188280
2,微増,0.000000,0.000000
3,用意,0.000000,0.081088
4,吸入,0.219793,0.352703
5,概説,0.000000,0.000000
6,作用,0.612037,0.587799
7,普及,0.094660,0.081088
8,梗塞,0.000000,0.000000
9,注射,0.314453,0.308730


### $H_P > \alpha H_N$ となる手がかり語を抽出

In [208]:
alpha = 2.
extract_clue_word_df = \
    word_entropy_df[
        (word_entropy_df.entropy_positive > (alpha * word_entropy_df.entropy_negative))
        & word_entropy_df.entropy_negative > 0
    ]
extract_clue_word_df

,clue_word,entropy_positive,entropy_negative
14,運動,0.189320,0.081088
20,適応,0.378639,0.081088
94,殺菌,0.350283,0.162176
112,手術,0.323575,0.074463
251,飲用,0.244692,0.081088
515,染色,0.366807,0.162176
625,固定,0.260314,0.128521
652,洗浄,0.283979,0.128521
668,認可,0.350283,0.128521
833,解熱,0.219793,0.081088


In [140]:
extract_clue_word_df.clue_word.to_csv("../data/clue_word_of_Sahen-setsuzoku_with_alpha=2.csv", index=False)

### $H_P > \beta + H_N$ となる手がかり語を抽出

In [209]:
extract_clue_word_df = \
    word_entropy_df[
        (word_entropy_df.entropy_positive > (0.10 + word_entropy_df.entropy_negative))
        & word_entropy_df.entropy_negative > 0
    ]
extract_clue_word_df.sort_values('entropy_positive', ascending=False)

,clue_word,entropy_positive,entropy_negative
727,使用,0.710212,0.575982
41,利用,0.674770,0.503511
1235,治療,0.630974,0.426740
453,添加,0.501420,0.308730
853,改善,0.418235,0.297146
20,適応,0.378639,0.081088
562,配合,0.373689,0.227642
429,鎮静,0.369825,0.227642
515,染色,0.366807,0.162176
668,認可,0.350283,0.128521


In [211]:
extract_clue_word_df.clue_word.to_csv("../data/clue_word_of_Sahen-setsuzoku_with_beta=0.1.csv", index=False)

## 利用・使用が含まれている文を調べる

In [212]:
word_entropy_df[word_entropy_df.clue_word == '利用']

,clue_word,entropy_positive,entropy_negative
41,利用,0.67477,0.503511


In [213]:
word_entropy_df[word_entropy_df.clue_word == '使用']

,clue_word,entropy_positive,entropy_negative
727,使用,0.710212,0.575982


In [10]:
extract_df = train.loc[train.sentence.str.contains(r'改善')]
extract_df.loc[extract_df.label == 1].head()

,sentence,label
5327,5q-症候群以外の骨髄異形成症候群でも、一部では貧血の改善が見られることがある。,1
5328,5q-症候群では貧血が強く輸血依存になることが多かったが、レナリドミドの服用で多くの例で貧血...,1
5547,ワラビなどの山菜のアク抜き、松の実などの臭み取り、豆を早く煮るため、肉を柔らかくする下ごしら...,1
5974,米国などでは、単独で短時間作用型の睡眠改善薬として用いられるほか、他の成分と組み合わせて風邪...,1
5976,ドキシラミンはエタノールアミン系抗ヒスタミン薬である コハク酸ドキシラミンの米国での睡眠改善...,1


In [11]:
[s['sentence'] for i, s in extract_df.loc[extract_df.label == 1].head(50).iterrows()]

['5q-症候群以外の骨髄異形成症候群でも、一部では貧血の改善が見られることがある。',
 '5q-症候群では貧血が強く輸血依存になることが多かったが、レナリドミドの服用で多くの例で貧血が改善し輸血が不要となっている。',
 'ワラビなどの山菜のアク抜き、松の実などの臭み取り、豆を早く煮るため、肉を柔らかくする下ごしらえ、グレープフルーツや夏みかんの強烈な酸味を中和させるために直接かけたり、冷凍エビの食感改善などにも使うことができる。',
 '米国などでは、単独で短時間作用型の睡眠改善薬として用いられるほか、他の成分と組み合わせて風邪やアレルギーの諸症状を緩和するための夜間服用型の薬として用いられることがある。',
 'ドキシラミンはエタノールアミン系抗ヒスタミン薬である コハク酸ドキシラミンの米国での睡眠改善薬としての用量は6.25mg～25mgである。',
 '（この比較については不確かであると指摘する専門家が多く、ましてや日本未発売の医薬品である）また、ドキシラミンは米国の店頭で市販される睡眠改善薬の中では最も効き目が強く、その強さは一部の処方箋が必要な睡眠薬を上回り、ドキシラミンの鎮静作用はフェノバルビタールより強いとする研究もある。',
 'またOTCの睡眠改善薬として、Somnil、Dozile、Donormyl、Restavit、Unisom-2などの商品名で用いられている。',
 'Unisomの商品名で販売されている錠剤タイプの睡眠改善薬の有効成分だが、Unisomの軟カプセル剤タイプではドキシラミンの代わりにジフェンヒドラミンが用いられている。',
 'また精神や肉体的ストレスを減少させ、カフェインとの相乗効果もあいまって認知活動や気分の改善もみられる。',
 '月経前症候群（PMS）に関しては、PMS時のイライラ、憂鬱、集中力の低下等の精神的症状を改善することが報告されている。',
 '中途覚醒の減少が認められたほか、被験者へのアンケートにより起床時の爽快感、熟眠感、疲労回復感の改善が認められている。',
 '睡眠に関しては、テアニン摂取により睡眠の質の改善が報告されている。',
 '老齢患者の栄養改善、火傷の治療などにおいて、治療の改善効果があるとの報告がある。',
 'また、テアニンは緑茶の呈味改善剤として使用されているが、緑茶以外の様々な